Data cleaning for lyrics. This is to apply language detection and '[]' bracket removal to a dataframe which already has lyrics, and further apply tokenization and sentiment scoring.

Works well, need to convert to .py script

In [ ]:
# Cleaning:
# Not all songs same length
# Not all have [Verse 1] or [Intro] as start
# -if using '[' as strip point
# -if not, can use 'Lyrics' as strip point
# if neither, leave it, and list locations (to check results)
# If they have a '[' in the lyrics, strip up to the first one
# words with dashes '-' or in parentheses '()' or #
# words in other languages

# Custom STOP words
# [Intro, Chorus, Verse #, Bridge, Interlude, x#, Pre-Chorus]

In [1]:
import pandas as pd
import langdetect

df = pd.read_csv("Recession_artists.csv", low_memory=False)
#df.drop(columns='lyrics_found')
#df.drop(columns='Unnamed: 0', inplace=True)
df.head()

,release_id,album_title,track_title,track_position,duration,year,country,genre,style,artist,source,lyrics
0,NaN,TiK ToK,TiK ToK,NaN,3:20,2009,US,Pop,NaN,Kesha,NaN,Wake up in the mornin’ feelin’ like P. Diddy (...
1,NaN,Ur So Gay,Ur So Gay,NaN,3:39,2007,US,Pop,NaN,Katy Perry,NaN,NaN
2,NaN,Ur So Gay,Use Your Love,NaN,3:03,2007,US,Pop,NaN,Katy Perry,NaN,NaN
3,NaN,Ur So Gay,Lost,NaN,4:20,2007,US,Pop,NaN,Katy Perry,NaN,NaN
4,NaN,One Of The Boys,One Of The Boys,NaN,4:07,2008,US,Pop,NaN,Katy Perry,NaN,[Intro]\nDo-do-do\nDo-do-do-do-do-do\nDo-do-do...


In [3]:
from langdetect import detect
def safe_detect(text):
    try:
        return detect(text)
    except:
        return "unknown"

df['language'] = df['lyrics'].apply(safe_detect)

In [ ]:
df_subset = df[df['lyrics'].notna()].copy()
df_subset.shape

In [ ]:
# may need to alter this choice, as there are multi-lingual tracks ending up in here
df_subset = df_subset[df_subset['language'] == 'en'].reindex()
df_subset.shape

In [ ]:
# Pre-check of troublemakers
print(df_subset['lyrics'].iloc[1], '\n-----')
print(df_subset['lyrics'].iloc[30], '\n-----')
print(df_subset['lyrics'].iloc[40], '\n-----')
print(df_subset['lyrics'].iloc[50], '\n-----')
print(df_subset['lyrics'].iloc[80])

In [5]:
import re

unmatched_lyrics = []

def clean_lyrics(text):
    if pd.isna(text):
        return text
    match = re.search(r'\[.*?\]', text)
    match1 = re.search(r'Lyrics', text)
    if match:
        return text[match.start():].strip()
    elif match1:
        return text[match1.end():].strip()
    else:
        unmatched_lyrics.append(text)
        return text.strip()  # Return original text if no bracketed section

#df_subset['clean_lyrics'] = df_subset['lyrics'].apply(clean_lyrics)
df['clean_lyrics'] = df['lyrics'].apply(clean_lyrics)

In [ ]:
# Re-check to confirm the troublemakers were cleaned
print("Number of tracks that don't meet either condition:", len(unmatched_lyrics))
print(df_subset['clean_lyrics'].iloc[1], '\n-----')
print(df_subset['clean_lyrics'].iloc[30], '\n-----')
print(df_subset['clean_lyrics'].iloc[40], '\n-----')
print(df_subset['clean_lyrics'].iloc[50], '\n-----')
print(df_subset['clean_lyrics'].iloc[80])

In [21]:
# Need to create a list of all bracketed words for unique stopword list
unique_brackets = set().union(*df['clean_lyrics'].dropna().apply(
    lambda x: set(re.findall(r'\[.*?\]', x))
))

print(f"Found {len(unique_brackets)} unique bracketed phrases:")
print(sorted(unique_brackets))

Found 73 unique bracketed phrases:
['[Break: Britney Spears]', '[Break]', '[Breakdown: Britney Spears & Avant]', '[Breakdown: Lady Gaga]', '[Breakdown]', '[Bridge: Britney Spears & Danja]', '[Bridge: Britney Spears]', '[Bridge: Danja & Britney Spears]', '[Bridge: Lady Gaga & Akon]', '[Bridge: Lady Gaga, Beyoncé & Both]', '[Bridge: Lady Gaga]', '[Bridge]', '[Chorus 1]', '[Chorus 2]', '[Chorus: Britney Spears & Danja]', '[Chorus: Britney Spears, Bloodshy & Robyn]', '[Chorus: Britney Spears]', '[Chorus: Danja & Britney Spears]', '[Chorus: Lady Gaga & Akon]', '[Chorus: Lady Gaga & Beyoncé, Lady Gaga]', '[Chorus: Lady Gaga & Space Cowboy]', '[Chorus: Lady Gaga]', '[Chorus: Lyrica Anderson]', '[Chorus]', '[Instrumental Break]', '[Interlude: Akon]', '[Interlude: Britney Spears & Danja]', '[Interlude]', '[Intro: Annie Lennox]', '[Intro: Avant]', '[Intro: Britney Spears]', '[Intro: Danja & Britney Spears]', '[Intro: Danja]', '[Intro: Lady Gaga & Akon]', '[Intro: Lady Gaga]', '[Intro]', '[Outro:

In [23]:
# oookay, looks like it will be better to remove them instead to have clean lyrics to work with.
# Now we need to figure out how to discard repetitions in lyrics, since they don't add anything to 
# change the sentiment or understanding.

df['clean_lyrics'] = (
    df['clean_lyrics']
    .str.replace(r'\[.*?\]', '', regex=True)         # Remove all bracketed phrases
    .str.replace(r'\s+', ' ', regex=True)            # Normalize extra spaces
    .str.strip()                                     # Trim leading/trailing spaces
)

In [25]:
# Double-check that it worked
check_brackets = set().union(*df['clean_lyrics'].dropna().apply(
    lambda x: set(re.findall(r'\[.*?\]', x))
))

print(f"Found {len(check_brackets)} bracketed phrases:")
print(sorted(check_brackets))

Found 0 bracketed phrases:
[]
